# 1

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!ls gdrive/MyDrive/datasets/ham10000

HAM10000_images_part_1	HAM10000_images_part_2	HAM10000_metadata.csv


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(0)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

In [3]:
import torch
import torchvision.datasets
from torchvision import transforms

In [14]:
X_train = torch.load('gdrive/MyDrive/datasets/ham_100_X_train.pt')
y_train = torch.load('gdrive/MyDrive/datasets/ham_100_y_train.pt')
X_test = torch.load('gdrive/MyDrive/datasets/ham_100_X_test.pt')
y_test = torch.load('gdrive/MyDrive/datasets/ham_100_y_test.pt')

In [5]:
X_train /= 255.
X_test /= 255.

In [15]:
X_train = torch.nn.functional.normalize(X_train)
X_test = torch.nn.functional.normalize(X_test)

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [16]:
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [7]:
import random

import torchvision.datasets
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [8]:
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks')

In [9]:
import pytorch_optimizer as optim

In [17]:
def train(net, X_train, y_train, X_test, y_test):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=3.0e-3)
    optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-4)
    #optimizer = optim.PNMBelief(net.parameters(), lr=0.001)
    #optimizer = optim.YogiPNM(net.parameters())
    #optimizer = optim.AdaBelief(net.parameters())
    #optimizer = optim.DiffGrad(net.parameters(), lr=0.5e-3)
    #optimizer = optim.Yogi(net.parameters())
    
    batch_size = 10

    test_accuracy_history = []
    test_loss_history = []

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    for epoch in range(14):
        print("epoch ", epoch)
        order = np.random.permutation(len(X_train))
        
        if epoch == 9:
            #optimizer = torch.optim.SGD(net.parameters(), lr=1.0e-4)
            optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-5)
            #optimizer = optim.PNMBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.YogiPNM(net.parameters(), lr=1.0e-4)
            #optimizer = optim.AdaBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.DiffGrad(net.parameters(), lr=1.0e-5)
            #optimizer = optim.Yogi(net.parameters(), lr=1.0e-4)
    

        for start_index in tqdm(range(0, len(X_train), batch_size)):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = net.forward(X_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            
            X_batch

        net.eval()
        with torch.no_grad():
            test_preds = net.forward(X_test)
            loss_value = loss(test_preds, y_test).item()
            test_loss_history.append(loss_value)

            accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().item()
            test_accuracy_history.append(accuracy)
            #print(test_preds.argmax(dim=1))
            #print(y_test)

        print("accuracy = ", accuracy)
        print("\n\n")

    return test_accuracy_history, test_loss_history

In [11]:
accuracies = {}
losses = {}

In [12]:
!pip install timm
import timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00


In [ ]:
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw

In [ ]:
model = timm.create_model('inception_v4', pretrained=True)
model.last_linear = torch.nn.Linear(in_features=1536, out_features=7)

accuracies['inception_v4+SGD'], losses['inception_v4+SGD'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [01:00<00:00, 13.29it/s]


accuracy =  0.6779830455780029



epoch  1


100%|██████████| 802/802 [01:01<00:00, 13.10it/s]


accuracy =  0.7139291167259216



epoch  2


...

accuracy =  0.7523714303970337



epoch  12


100%|██████████| 802/802 [01:00<00:00, 13.23it/s]


accuracy =  0.7508736848831177



epoch  13


100%|██████████| 802/802 [01:01<00:00, 13.05it/s]


accuracy =  0.7443834543228149





In [19]:
model = timm.create_model('inception_v4', pretrained=True)
model.last_linear = torch.nn.Linear(in_features=1536, out_features=7)


accuracies['inception_v4+Adam'], losses['inception_v4+Adam'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [01:17<00:00, 10.38it/s]


accuracy =  0.6200699210166931



epoch  1


100%|██████████| 802/802 [01:14<00:00, 10.72it/s]


accuracy =  0.6250624060630798



epoch  2


...

accuracy =  0.6610084772109985



epoch  6


100%|██████████| 802/802 [01:16<00:00, 10.45it/s]


accuracy =  0.6225661635398865



epoch  7


 15%|█▌        | 122/802 [00:11<01:04, 10.48it/s]


KeyboardInterrupt: ignored

In [ ]:
model = timm.create_model('inception_v4', pretrained=True)
model.last_linear = torch.nn.Linear(in_features=1536, out_features=7)

accuracies['inception_v4+Yogi'], losses['inception_v4+Yogi'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [01:28<00:00,  9.07it/s]


accuracy =  0.6814777851104736



epoch  1


100%|██████████| 802/802 [01:28<00:00,  9.11it/s]


accuracy =  0.7244133949279785



epoch  2


...

accuracy =  0.7343984246253967



epoch  12


100%|██████████| 802/802 [01:27<00:00,  9.19it/s]


accuracy =  0.7403894066810608



epoch  13


100%|██████████| 802/802 [01:28<00:00,  9.06it/s]


accuracy =  0.7433848977088928





In [ ]:
model = timm.create_model('inception_v4', pretrained=True)
model.last_linear = torch.nn.Linear(in_features=1536, out_features=7)

accuracies['inception_v4+DiffGrad'], losses['inception_v4+DiffGrad'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [02:05<00:00,  6.37it/s]


accuracy =  0.7024463415145874



epoch  1


100%|██████████| 802/802 [02:04<00:00,  6.43it/s]


accuracy =  0.7234148979187012



epoch  2


...

accuracy =  0.7538691759109497



epoch  12


100%|██████████| 802/802 [02:10<00:00,  6.17it/s]


accuracy =  0.7598602175712585



epoch  13


100%|██████████| 802/802 [02:07<00:00,  6.28it/s]


accuracy =  0.7558662295341492





In [ ]:
model = timm.create_model('inception_v4', pretrained=True)
model.last_linear = torch.nn.Linear(in_features=1536, out_features=7)

accuracies['inception_v4+PNMBelief'], losses['inception_v4+PNMBelief'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 802/802 [01:56<00:00,  6.89it/s]


accuracy =  0.7234148979187012



epoch  1


100%|██████████| 802/802 [01:55<00:00,  6.97it/s]


accuracy =  0.717923104763031



epoch  2


...

accuracy =  0.784323513507843



epoch  12


100%|██████████| 802/802 [01:54<00:00,  7.02it/s]


accuracy =  0.7883175015449524



epoch  13


100%|██████████| 802/802 [01:55<00:00,  6.96it/s]


accuracy =  0.7903145551681519





# ansamble

In [ ]:
APreds = pd.DataFrame()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [ ]:
APreds['SGD'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds['Adam'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds['Yogi'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds['DiffGrad'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds['PNMBelief'] = test_preds.argmax(dim=1).tolist()

In [ ]:
APreds

,Adam,SGD,Yogi,DiffGrad,PNMBelief
0,4,4,4,4,4
1,4,4,4,4,4
2,4,4,4,4,4
3,4,2,2,2,4
4,4,4,1,4,1
...,...,...,...,...,...
1998,4,4,4,4,4
1999,5,4,4,4,4
2000,4,4,4,4,4
2001,4,4,4,4,4


In [ ]:
APreds = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_resnext.csv')
APreds = APreds.drop(['Unnamed: 0'], axis=1)

In [ ]:
APreds.to_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_resnext.csv')

In [ ]:
ARes = pd.DataFrame(np.zeros((2003,7), dtype=int))

for i in range(2003):
    for j in range(5):
        if j == 0:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.81477
        if j == 1:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.74438
        if j == 2:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.7433
        if j == 3:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.7558
        if j == 4:
            ARes.iloc[i, APreds.iloc[i, j]] += 0.79031

In [ ]:
(torch.tensor(ARes.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8122815489768982

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(test_preds.argmax(dim=1).tolist(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #Adam

In [ ]:
D #PNMBelief

In [ ]:
D #Yogi

In [ ]:
D #DiffGrad

In [ ]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(ARes.idxmax(axis=1).to_list(), y_test.tolist()):
    D[y][x] += 1

In [ ]:
D #ansamble

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
precision_score(y_test, test_preds.argmax(dim=1), average='micro')


In [ ]:
print('precision: ', precision_score(y_test, test_preds.argmax(dim=1), average=None))
print('recall: ', recall_score(y_test, test_preds.argmax(dim=1), average=None))
print('f1_score: ', f1_score(y_test, test_preds.argmax(dim=1), average=None))

# SoftMax

In [ ]:
ARes2 = pd.DataFrame(np.zeros((2003,7), dtype=int))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [ ]:
m = torch.nn.Softmax(dim=1)

In [ ]:
test_preds = m(test_preds) * 0.74438 #SGD

In [ ]:
test_preds = m(test_preds) * 0.81477 #Adam

In [ ]:
test_preds = m(test_preds) * 0.7433 #Yogi

In [ ]:
test_preds = m(test_preds) * 0.7558 #DiffGrad

In [ ]:
test_preds = m(test_preds) * 0.79031 #PNMBelief

In [ ]:
ARes2 += pd.DataFrame(test_preds.to('cpu'))

In [ ]:
test_preds

In [ ]:
ARes2

,0,1,2,3,4,5,6
0,0.000047,0.000663,0.000139,0.002047,0.811565,0.000072,0.000237
1,0.000039,0.000069,0.006492,0.000019,0.807606,0.000504,0.000043
2,0.000000,0.000000,0.000000,0.000000,0.814770,0.000000,0.000000
3,0.000495,0.001883,0.202653,0.000357,0.604376,0.003847,0.001158
4,0.002271,0.001007,0.019525,0.000198,0.790598,0.000913,0.000257
...,...,...,...,...,...,...,...
1998,0.000122,0.000735,0.001966,0.000144,0.804362,0.007273,0.000169
1999,0.002705,0.002380,0.074427,0.001507,0.125165,0.607663,0.000924
2000,0.000025,0.000137,0.019050,0.000011,0.795142,0.000153,0.000252
2001,0.000000,0.000000,0.000000,0.000000,0.814770,0.000000,0.000000


In [ ]:
(torch.tensor(ARes2.idxmax(axis=1)).to(device) == y_test).float().mean().item()

0.8037943243980408